<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

Leonardo Rubio Salcedo

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
datos = pd.read_csv(path)
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [12]:
from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    #texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    #texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto.split() ])
    return texto

In [14]:
datos["pp"]= datos['review_body'].apply(lambda texto: pre_procesado(texto))
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [15]:
# Crear una representación de los documentos en forma de diccionario
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint

In [16]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(datos['pp'].values)
dictionary

In [49]:
# Filtrar palabras muy frecuentes y poco frecuentes
dictionary.filter_extremes(no_below=20, no_above=0.4)
corpus = [dictionary.doc2bow(text) for text in datos['pp'].values]

In [56]:
#Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=200)

###  `[25 pts]` Punto 3: Visualización de LDA

In [57]:
model.print_topics(7)

[(0,
  '0.075*"calidad" + 0.060*"precio" + 0.046*"bien" + 0.035*"si" + 0.034*"buena" + 0.033*"bonito" + 0.028*"bastante" + 0.027*"foto" + 0.027*"mando" + 0.026*"puede"'),
 (1,
  '0.066*"juego" + 0.059*"producto" + 0.055*"bien" + 0.033*"perfecto" + 0.031*"buen" + 0.028*"llegado" + 0.027*"día" + 0.026*"caja" + 0.026*"si" + 0.025*"solo"')]

In [58]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)